In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.quantum_info import Statevector

In [11]:
from sqlite_mps import SQLITE_MPS

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def get_qiskit_statevector(qiskit_circ) -> Statevector:
        circ = qiskit_circ.copy().reverse_bits()
        circ.save_statevector()
        # Transpile for simulator
        simulator = AerSimulator(method="statevector")
        circ = transpile(circ, simulator)
        # Run and get statevector
        result = simulator.run(circ).result()
        statevector = result.get_statevector(circ)
        return statevector

In [8]:
def measure_qiskit_probablities(qiskit_circ):
        statevector = get_qiskit_statevector(qiskit_circ)
        probs = statevector.probabilities(decimals=5)
        return np.array(statevector)

In [38]:
qiskit_circ = QuantumCircuit(3)
qiskit_circ.h(0)
qiskit_circ.p(0.244444444,0)
print(measure_qiskit_probablities(qiskit_circ))

[0.70710678+0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.68608586+0.17113209j 0.        +0.j
 0.        +0.j         0.        +0.j        ]


In [36]:
sim=SQLITE_MPS(3,{"H","CNOT"})
sim.apply_one_qbit_gate(0,"h")
sim.apply_one_qbit_gate(1,"h")
print(sim.get_statevector_np())

0.00016458379104733467
0.00038966676220297813
[0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j]
